# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

$Gini(D_i) = 1 - \sum_{j=1}^xP_j^2$

In [123]:
def get_gini(df, label):
    # df 데이터의 개수
    n = df.shape[0]
    
    # label의 정답 값(꼭 yes나 no라는 보장이 없으므로)
    label_value = np.unique(df[label])
    
    # df의 label 컬럼 중 첫번째 레이블 수
    label1 = np.sum(df[label] == label_value[0])
    
    # df의 label 컬럼 중 두번째 레이블 수(2번째 까지만 고려, binary classification)
    label2 = n - label1 # 첫번째 레이블 갯수 + 두번째 레이블 갯수 = 전체 데이터 갯수
    
    # 지니 계수 계산
    gini = 1 - (label1 / n) ** 2 - (label2 / n) ** 2
    
    return gini    

In [115]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [116]:
from itertools import combinations

def get_binary_split(df, attribute):
    # 결과를 담을 리스트
    result = []
    
    # 입력 df의 attribute 컬럼의 unique 값들을 추출
    unique_value = np.unique(df[attribute])
    
    # combinations를 이용해 원소가 1, unique한 원소 개수보다 1개 작은 숫자까지 조합 생성
    # unique한 갯수가 n이라고 가정하면 1<->n-1, 2<->n-2, ... 꼴로 매칭이 되는 쌍들이 나와야함.(2^(n-1) - 1개)
    for i in range(1, len(unique_value)):
        # unique한 값들을 가지고 1개 또는 2개의 원소를 갖도록 조합 실행
        for att in combinations(unique_value, i):
            result.append(list(att))
    
    return result

In [117]:
get_binary_split(pd_data,"age")

[['middle_aged'],
 ['senior'],
 ['youth'],
 ['middle_aged', 'senior'],
 ['middle_aged', 'youth'],
 ['senior', 'youth']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [181]:
def get_attribute_gini_index(df, attribute, label):
    # DataFrame의 obs 갯수
    n = df.shape[0]
    
    # 일단 경우의 수로 split
    split = get_binary_split(df, attribute)
    
    # 결과를 담을 dictionary 선언
    result = {}
    
    # split에 담겨있는 원소 하나씩 수행
    for col in split:
        # dictionary에 담을 이름 선언(2개 이상일시 _로 구분되고 1개일 시 그냥 들어갈 것.)
        col_name = "_".join(col)
        
        # 해당 attribute가 col에 속하는가??
        # 뒤집기 연산 (~)을 하기위해 numpy array 형식으로 지정.
        mask = np.array(list(map(lambda x : x in col, df[attribute])))
        
        # 원하는 DataFrame만 뽑아주기(t = target, r = rest)
        data_t = df.loc[mask, :]
        data_r = df.loc[~mask, :]        

        # gini계수 계산
        gini_t = get_gini(data_t, label)
        gini_r = get_gini(data_r, label)
        
        gini = len(data_t) / n * gini_t + len(data_r) / n * gini_r
        
        # dictionary에 추가
        result[col_name] = gini
        
    return result

In [131]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth': 0.3936507936507937,
 'middle_aged_senior': 0.3936507936507937,
 'middle_aged_youth': 0.45714285714285713,
 'senior_youth': 0.35714285714285715}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [132]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [133]:
##문제1 답안
# income으로 이진 분류 조합 결과 출력
get_binary_split(pd_data,"income")

[['high'],
 ['low'],
 ['medium'],
 ['high', 'low'],
 ['high', 'medium'],
 ['low', 'medium']]

- unique value가 총 3개라 2개(3-1개)의 원소로 구성된 리스트 생성.
- $2^(3-1) - 1 = 3쌍

In [185]:
##문제2 답안
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행
for col in features:
    # 해당 변수 중 가장 낮은 gini 계수와 변수 출력
    min_gini = min(get_attribute_gini_index(pd_data, col, label).items())[1]
    print(f"Minimum Gini index of {col} : {min_gini:.4f}")

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


##### - 확인 결과 age 변수의 지니 계수가 0.3571로 가장 낮으므로 주어진 변수 중 가장 불순도를 낮추면서 순도를 증가시키는 방향으로 분류를 잘 할 수 있음.

In [186]:
##문제3 답안
# age 변수를 어떤 attribute를 기준으로 나눌지 확인
age_division = get_attribute_gini_index(pd_data, "age", label)
sorted(age_division.items(), key=lambda x:x[1])

[('middle_aged', 0.35714285714285715),
 ('senior_youth', 0.35714285714285715),
 ('youth', 0.3936507936507937),
 ('middle_aged_senior', 0.3936507936507937),
 ('senior', 0.45714285714285713),
 ('middle_aged_youth', 0.45714285714285713)]

#### middle_aged VS senior, youth 로 나누는 것이 좋아보임.

In [187]:
middle = ['middle_aged']
# age 변수가 middle_aged인 변수를 뽑아내기 위한 마스크
mask = np.array([True if x in middle else False for x in pd_data['age']])

# DataFrame middle_aged 추출
df_m = pd_data.loc[mask, :]
# DataFrame Senior, youth 추출
df_sy = pd_data.loc[~mask, :]

In [193]:
# 변수명 중에서 age 변수 제거
features.remove('age')

# Middle_aged DataFrame
# 각 변수를 대상으로 반복문 수행
for col in features:
    # 해당 변수 중 가장 낮은 gini 계수와 변수 출력
    min_gini = min(get_attribute_gini_index(df_m, col, label).items())[1]
    print(f"Age : Middle_aged, Minimum Gini index of {col} : {min_gini:.4f}")

print("#############################################################")
# Youth, Senior DataFrame
# 각 변수를 대상으로 반복문 수행
for col in features:
    # 해당 변수 중 가장 낮은 gini 계수와 변수 출력
    min_gini = min(get_attribute_gini_index(df_sy, col, label).items())[1]
    print(f"Age : Middle_aged, Minimum Gini index of {col} : {min_gini:.4f}")


Age : Middle_aged, Minimum Gini index of income : 0.0000
Age : Middle_aged, Minimum Gini index of student : 0.0000
Age : Middle_aged, Minimum Gini index of credit_rating : 0.0000
#############################################################
Age : Middle_aged, Minimum Gini index of income : 0.3750
Age : Middle_aged, Minimum Gini index of student : 0.3200
Age : Middle_aged, Minimum Gini index of credit_rating : 0.4167


In [194]:
df_m

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


##### Middle_aged DataFrame의 경우에는 label이 전부 yes라 순도가 1이라 더이상 분류 필요 x

In [195]:
df_sy

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


##### Senior, Youth DataFrame의 경우 student 변수의 gini index가 0.32로 나머지 변수에 비해 가장 낮음.